In [ ]:
from setups_paper_gwp import *
from copy import deepcopy
from gsa_framework.sensitivity_analysis.saltelli_sobol import *

In [2]:
# read X and Y
num_params = 10000
iter_salt = 40*num_params
gsa = setup_salt(num_params, iter_salt, setup_lca_model_paper)
Y = read_hdf5_array(gsa.filepath_Y).flatten()

num_steps = 50
num_bootstrap = 10

# Convergence class
conv = Convergence(
    gsa.filepath_Y,
    gsa.num_params,
    gsa.generate_gsa_indices,
    gsa.gsa_label,
    gsa.write_dir,
    num_steps=num_steps,
)

# Generate random seeds
np.random.seed(gsa.seed)
stability_seeds = np.random.randint(
    low=0,
    high=2147483647,
    size=(len(conv.iterations_for_convergence), num_bootstrap),
)

In [18]:
%%time
def sobol_first_order(A, AB, B):
    """First order estimator normalized by sample variance."""
#     return np.mean(B * (AB - A), axis=0) / np.var(np.r_[A, B], axis=0)
    return np.mean(B * (AB - A), axis=0) # in the paper and in SALib
    
# def sobol_indices_stability(Y, num_params, iterations_for_convergence, num_bootstrap):    
for iterations_current in conv.iterations_for_convergence:
    iterations_per_param = iterations_current // (num_params+2)
    print(iterations_per_param)
#     Ycurrent = Y[:iterations_current]
    A, B, AB = separate_output_values(Y, num_params)
    choice = np.random.randint(iterations_per_param, size=(iterations_per_param, num_bootstrap))
    A_choice = A[choice]
    B_choice = B[choice]
    first = np.zeros((num_bootstrap,num_params))
    total = np.zeros((num_bootstrap,num_params))
    for j in range(num_params):
        print(j)
        first[:,j] = sobol_first_order(A_choice, AB[choice, j], B_choice)
        total[:,j] = sobol_total_order(A_choice, AB[choice, j], B_choice)
    S_dict = {
        iterations_current: {
            "First order": first,
            "Total order": total,
        }
    }
#     return S_dict
    

1
0


ValueError: could not broadcast input array from shape (10,10) into shape (10)

In [19]:
sobol_first_order(A_choice, AB[choice, j], B_choice)



array([[10.04216177, 10.04216177, 10.04216177, 10.04216177, 10.04216177,
        10.04216177, 10.04216177, 10.04216177, 10.04216177, 10.04216177],
       [10.04216177, 10.04216177, 10.04216177, 10.04216177, 10.04216177,
        10.04216177, 10.04216177, 10.04216177, 10.04216177, 10.04216177],
       [10.04216177, 10.04216177, 10.04216177, 10.04216177, 10.04216177,
        10.04216177, 10.04216177, 10.04216177, 10.04216177, 10.04216177],
       [10.04216177, 10.04216177, 10.04216177, 10.04216177, 10.04216177,
        10.04216177, 10.04216177, 10.04216177, 10.04216177, 10.04216177],
       [10.04216177, 10.04216177, 10.04216177, 10.04216177, 10.04216177,
        10.04216177, 10.04216177, 10.04216177, 10.04216177, 10.04216177],
       [10.04216177, 10.04216177, 10.04216177, 10.04216177, 10.04216177,
        10.04216177, 10.04216177, 10.04216177, 10.04216177, 10.04216177],
       [10.04216177, 10.04216177, 10.04216177, 10.04216177, 10.04216177,
        10.04216177, 10.04216177, 10.04216177

In [11]:
A_choice


array([[[252.28081373],
        [252.28081373],
        [252.28081373],
        [252.28081373],
        [252.28081373],
        [252.28081373],
        [252.28081373],
        [252.28081373],
        [252.28081373],
        [252.28081373]]])

In [ ]:
AB[choice]

In [ ]:
conv.iterations_min, conv.iterations_least_common_multiple

In [ ]:
%%time
filename_S = "stability.S.{}.{}.{}Step{}.{}.{}.pickle".format(
    gsa.gsa_label, gsa.sampling_label, gsa.iterations, conv.iterations_step, num_bootstrap, gsa.seed,
)
filepath_S = gsa.write_dir / "arrays" / filename_S
if filepath_S.exists():
    print("--> {} already exists".format(filename_S))
    S_dict_stability = read_pickle(filepath_S)
else:
    S_dict_stability = {}
    for i,iterations_current in enumerate(conv.iterations_for_convergence):
        S_array = np.zeros([0,num_params])
        print("{}".format(iterations_current))
        for j in range(num_bootstrap):
            stability_seed = stability_seeds[i,j]
            np.random.seed(stability_seed)
            
            
            choice = np.random.choice(np.arange(gsa.iterations), iterations_current, replace=False)
            Y_current = Y[choice]
            S_current = corrcoef_parallel_stability_spearman(Y_current, X_current)
            S_array = np.vstack([S_array, S_current])
        S_dict = {iterations_current: {"spearman": S_array}}
        S_dict_stability.update(S_dict)
    write_pickle(S_dict_stability, filepath_S)

